[Reference](https://medium.com/@parttimebloggers/metaclasses-in-python-3f82afd83064)

In [1]:
class Metaclass(type):
    pass

class Person(metaclass=Metaclass):
    pass

In [2]:
class Metaclass(type):
   def __new__(mcs, name, bases, namespace, **kwargs):
       return super().__new__(mcs, name, bases, namespace)

In [3]:
class M(type):
    def __new__(mcs, name, bases, namespace, **kwargs):
        # namespace contains all the attributes defined in the class including
        # variables, methods and module it was defined in.
        if "some_important_func" not in namespace:
            raise NotImplementedError("some_important_func has to be implemented in subclasses of this function")
        return super(M, mcs).__new__(mcs, name, bases, namespace, **kwargs)


class Base(metaclass=M):
    def some_important_func(self):
        pass


class Point(Base): # this will pass the check
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

    def some_important_func(self):
        return f"Point(x={self.x}, y={self.y})"

class Line(Base): # this will raise an error because some_important_func is not defined in this class
    def __init__(self, points: list):
        self.points = points

NotImplementedError: some_important_func has to be implemented in subclasses of this function

In [6]:
import inspect
import types

class MetaClass(type):
    def __new__(mcs, name, bases, namespace, **kwargs):
        for attr, value in namespace.items():
            if type(value) is types.FunctionType:
                for param_key,param_val in inspect.signature(value).parameters.items():
                    if param_key not in ("self", "cls") and param_val.annotation == inspect._empty:
                        # raising exception
                        raise Exception(f"Class: {name} doesn't have annotations for method: {attr}")
        return super().__new__(mcs, name, bases, namespace)

class Base(metaclass=MetaClass):
    pass

class Fruit(Base):
    def __init__(self, name: str, price: float): # This will pass the annotation check
        self.name = name
        self.price = price

    def get_price(self, count=1): # this will raise exception because count doesn't have type annotation
        return count * self.price


Exception: Class: Fruit doesn't have annotations for method: get_price

In [7]:
def _make_init(annotations: dict):
    """
    returns the __init__ function from the given annotations in string form
    """
    return (
        f"def __init__(self, {','.join(annotations.keys())}):" +
        "".join(f"self.{_var} = {_var};" for _var in annotations.keys())
    )

class CodeGenerationMeta(type):
    def __new__(mcs, name, bases, namespace, **kwargs):
        annotations = namespace.get('__annotations__')
        _init = namespace.get('__init__')
        if annotations and not _init:
            # exec will execute the init str and add it to namespace
            exec(_make_init(annotations), globals(), namespace)
        return super().__new__(mcs, name, bases, namespace, **kwargs)

class DataClass(metaclass=DataClassMeta):
    pass

class Point(DataClass):
    x: int
    y: int
    z: int = 10

    def show(self):
        print(f"Point(x={self.x}, y={self.y}, {self.z})")

Point(1, 2, 3)

NameError: name 'DataClassMeta' is not defined

In [8]:
class NamespaceCustomizationType(type):
    @classmethod
    def __prepare__(mcs, name, bases):
        print(f"NamespaceCustomization.__prepare__ => mcs: {mcs} name: {name}: bases: {bases}")
        extra_attrs = super().__prepare__(mcs, name, bases)
        return {
            **extra_attrs,
            "_count": 0,
            "swear": lambda self: print("Watch your profanity!")
        }

    # __call__ function is called when creating a cls instance
    def __call__(cls, *args, **kwargs):
        cls._count += 1
        return cls(*args, **kwargs)

class Fruit(metaclass=NamespaceCustomizationType):
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price

f = Fruit("Mango", 10)
print(f.name)
print(f.price)
print(f._count) # this is also here from the __prepare__ method
print(f.swear())

NamespaceCustomization.__prepare__ => mcs: <class '__main__.NamespaceCustomizationType'> name: Fruit: bases: ()


RecursionError: maximum recursion depth exceeded while calling a Python object

In [9]:
class Metaclass(type):
    def __call_(cls, *args, **kwargs):
        return cls(*args, **kwargs)

In [10]:
class SingletonType(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        print("In SingletonType().__call__")
        if cls not in cls._instances:
            cls._instances[cls] = cls(*args, **kwargs)
        return cls._instances[cls]

class Point(metaclass=SingletonType):
    def __init__(self):
        self.x = 10
        self.y = 20

    def show(self):
        print(f"Point(x={self.x}, y={self.y})")

print(Point() is Point())

In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType().__call__
In SingletonType()._

RecursionError: maximum recursion depth exceeded

In [11]:
class Potato:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return f"Hi there! My name is {self.name}"

class PotatoType(type):
    def __call__(cls, *args, **kwargs):
        return Potato("Brown Potato")

class Base(metaclass=PotatoType):
    pass

class Person(Base):
    def __init__(self, name, age):
        self.name = name
        self.age = age

class Square(Base):
    def __init__(self, length):
        self.length = length

p = Person("Diwash", 24) # returns a potato object
s = Square(10) # returns a potato object
print(p)
print(s)

Hi there! My name is Brown Potato
Hi there! My name is Brown Potato


In [12]:
class Metaclass(type):
    def make(cls, *args, **kwargs):
        """
        Usually in a instance method of class, self or the instance gets passed
        by default as the first argument.  In the same way, in metaclasses,
        classes gets passed as the first argument to the instance method of a
        metaclass.  cls is like self here. (Classes are also called
        meta-instances.)
        """
        return cls(*args, **kwargs)

class Human(metaclass=Metaclass):
    pass

if __name__ == "__main__":
    Human.make()